

# Optimizing Computer Vision Applications

This tutorial shows some techniques to get better performance for computer vision applications with the Intel® Distribution of OpenVINO™ toolkit.



## 1. Setup the environment variables,download model files and import dependencies

In [ ]:
from IPython.display import HTML
import os
import time
import sys                                     
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent.parent.parent))
from demoTools.demoutils import *

In [ ]:
!/opt/intel/openvino/bin/setupvars.sh

In [ ]:
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name mobilenet-ssd -o models
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name ssd300 -o models
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name ssd512 -o models

In [ ]:
!mkdir -p models/SSD512/{FP16,FP32} 
!mkdir -p models/SSD300/{FP16,FP32}

### Run Model Optimizer on the models to get IR files

First, we will create the required directories, then run the model Optimizer to get the IR files. 

In [ ]:
! python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo_caffe.py --input_model models/public/mobilenet-ssd/mobilenet-ssd.caffemodel -o models/mobilenet-ssd/FP32/ --scale 256 --mean_values [127,127,127]
! python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo_caffe.py --input_model models/public/mobilenet-ssd/mobilenet-ssd.caffemodel -o models/mobilenet-ssd/FP16/ --scale 256 --mean_values [127,127,127] --data_type FP16
! python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo_caffe.py --input_model models/public/ssd300/models/VGGNet/VOC0712Plus/SSD_300x300_ft/VGG_VOC0712Plus_SSD_300x300_ft_iter_160000.caffemodel --input_proto models/public/ssd300/models/VGGNet/VOC0712Plus/SSD_300x300_ft/deploy.prototxt  -o models/SSD300/FP32/
! python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo_caffe.py --input_model models/public/ssd300/models/VGGNet/VOC0712Plus/SSD_300x300_ft/VGG_VOC0712Plus_SSD_300x300_ft_iter_160000.caffemodel --input_proto models/public/ssd300/models/VGGNet/VOC0712Plus/SSD_300x300_ft/deploy.prototxt -o models/SSD300/FP16/ --data_type FP16
! python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo_caffe.py --input_model models/public/ssd512/models/VGGNet/VOC0712Plus/SSD_512x512/VGG_VOC0712Plus_SSD_512x512_iter_240000.caffemodel --input_proto models/public/ssd512/models/VGGNet/VOC0712Plus/SSD_512x512/deploy.prototxt -o models/SSD512/FP32/
! python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo_caffe.py --input_model models/public/ssd512/models/VGGNet/VOC0712Plus/SSD_512x512/VGG_VOC0712Plus_SSD_512x512_iter_240000.caffemodel --input_proto models/public/ssd512/models/VGGNet/VOC0712Plus/SSD_512x512/deploy.prototxt -o models/SSD512/FP16/ --data_type FP16



## 2. Tune parameters - set batch size

In this section, we will see how changes in the batch size affect the performance. 

### Let us first look at the performance numbers for the batch size 1.
The default batch size for the Model Optimizer is 1.

In [ ]:
! python3 tutorial1_batch.py -i cars_1900.mp4 -m models/mobilenet-ssd/FP32/mobilenet-ssd.xml -l /opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/libcpu_extension_avx2.so


    
### Change the batch size to 2 and run the object-detection example for new batch size

In [ ]:
! python3 tutorial1_batch.py -i cars_1900.mp4 -m models/mobilenet-ssd/FP32/mobilenet-ssd.xml -l /opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/libcpu_extension_avx2.so  -b 2 


    
### Run the example for different batch sizes

Change the batch sizes to 8,16,32,64,128 and so on and see the performance difference in terms of the inference time.

## 3. Pick the right model based on application and hardware

Use/train a model with the right performance/accuracy tradeoffs. Performance differences between models can be bigger than any optimization you can do at the inference app level. Run various SSD models from the model_downloader in the car detection example which we used in the initial tutorial and observe the performance. We will run these tests on different hardware accelerators to determine how application performance depends on models as well as hardware.

In the previous step we have all the models convered and ready by model Optimizer. 

### Set environmental variables

In [ ]:
os.environ["VIDEO"] = "cars_1900.mp4"

### Create Job Script 

We will run the workload on several DevCloud's edge compute nodes. We will send work to the edge compute nodes by submitting jobs into a queue. For each job, we will specify the type of the edge compute server that must be allocated for the job.

To pass the specific variables to the Python code, we will use following arguments:

* `-m`&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;location of the optimized models XML
* `-i`&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;location of the input video
* `-r`&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;output directory
* `-d`&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;hardware device type (CPU, GPU, MYRIAD, HDDL or HETERO:FPGA,CPU)
* `-n`&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;number of infer requests

The job file will be executed directly on the edge compute node.

In [ ]:
%%writefile object_detection.sh

ME=`basename $0`

# The default path for the job is your home directory, so we change directory to where the files are.
cd $PBS_O_WORKDIR

# Object detection script writes output to a file inside a directory. We make sure that this directory exists.
# The output directory is the first argument of the bash script
while getopts 'd:f:i:r:n:m:x:?' OPTION; do
    case "$OPTION" in
    d)
        DEVICE=$OPTARG
        echo "$ME is using device $OPTARG"
      ;;

    i)
        INPUT_FILE=$OPTARG
        echo "$ME is using input file $OPTARG"
      ;;
    r)
        RESULTS_BASE=$OPTARG
        echo "$ME is using results base $OPTARG"
      ;;
    n)
        NUM_INFER_REQS=$OPTARG
        echo "$ME is running $OPTARG inference requests"
      ;;
    m)
        NUM_MODEL_PATH=$OPTARG
        echo "$ME is running $OPTARG inference requests"
      ;;
    esac  
done

RESULTS_PATH="${RESULTS_BASE}"
mkdir -p $RESULTS_PATH
echo "$ME is using results path $RESULTS_PATH"
 
# Running the object detection code
SAMPLEPATH=$PBS_O_WORKDIR
python3 tutorial1.py                        -m $NUM_MODEL_PATH \
                                            -i $INPUT_FILE \
                                            -o $RESULTS_PATH \
                                            -d $DEVICE \
                                            -nireq $NUM_INFER_REQS \
                                            -ce /opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/libcpu_extension_avx2.so

g++ -std=c++14 ROI_writer.cpp -o ROI_writer_${PBS_JOBID}  -lopencv_core -lopencv_videoio -lopencv_imgproc -lopencv_highgui  -fopenmp -I/opt/intel/openvino/opencv/include/ -L/opt/intel/openvino/opencv/lib/
# Rendering the output video
SKIPFRAME=1
RESOLUTION=0.5
./ROI_writer_${PBS_JOBID} $INPUT_FILE $RESULTS_PATH $SKIPFRAME $RESOLUTION
rm -f ROI_writer_${PBS_JOBID}

    
### Run the object detection example with different models on different devices.

For simplicity of the code and in order to put more focus on the performance number, video rendering with rectangle boxes for detected objects has been separated from object detection example(tutorial1.py). The inference difference in different scenarios can be seen in the progress bar after running the sample. 


### a) CPU

#### - Inferencing using **mobilenet-ssd** model

In [ ]:
#Submit job to the queue
job_id_core = !qsub object_detection.sh -l nodes=1:idc001skl:tank-870:i5-6500te -F "-r results/Core/mobilenet -d CPU -i $VIDEO -m models/mobilenet-ssd/FP32/mobilenet-ssd.xml -n 2" -N obj_det_core
print(job_id_core[0]) 
#Progress indicators
if job_id_core:
    progressIndicator('results/Core/mobilenet', 'pre_progress.txt', "Preprocessing", 0, 100)
    progressIndicator('results/Core/mobilenet', 'i_progress.txt', "Inference", 0, 100)
    progressIndicator('results/Core/mobilenet', 'post_progress.txt', "Rendering", 0, 100)

#### - Inferencing using **ssd300** model

In [ ]:
#Submit job to the queue
job_id_core = !qsub object_detection.sh -l nodes=1:idc001skl:i5-6500te -F "-r results/Core/ssd300 -d CPU -i $VIDEO -n 2 -m models/SSD300/FP32/VGG_VOC0712Plus_SSD_300x300_ft_iter_160000.xml" -N obj_det_core
print(job_id_core[0]) 
#Progress indicators
if job_id_core:
    progressIndicator('results/Core/ssd300', 'pre_progress.txt', "Preprocessing", 0, 100)
    progressIndicator('results/Core/ssd300', 'i_progress.txt', "Inference", 0, 100)
    progressIndicator('results/Core/ssd300', 'post_progress.txt', "Rendering", 0, 100)

#### - Inferencing using **ssd512** model

In [ ]:
#Submit job to the queue
job_id_core = !qsub object_detection.sh -l nodes=1:idc001skl:i5-6500te -F "-r results/Core/ssd512 -d CPU -i $VIDEO -n 2 -m models/SSD512/FP32/VGG_VOC0712Plus_SSD_512x512_iter_240000.xml" -N obj_det_core
print(job_id_core[0]) 
#Progress indicators
if job_id_core:
    progressIndicator('results/Core/ssd512', 'pre_progress.txt', "Preprocessing", 0, 100)
    progressIndicator('results/Core/ssd512', 'i_progress.txt', "Inference", 0, 100)
    progressIndicator('results/Core/ssd512', 'post_progress.txt', "Rendering", 0, 100)


    
### b) GPU

#### - Inferencing using **mobilenet-ssd** model

In [ ]:
#Submit job to the queue
job_id_gpu = !qsub object_detection.sh -l nodes=1:idc001skl:intel-hd-530 -F "-r results/GPU/mobilenet -d GPU -i $VIDEO -m models/mobilenet-ssd/FP32/mobilenet-ssd.xml -n 4" -N obj_det_gpu 
print(job_id_gpu[0]) 
#Progress indicators
if job_id_gpu:
    progressIndicator('results/GPU/mobilenet', 'pre_progress.txt', "Preprocessing", 0, 100)
    progressIndicator('results/GPU/mobilenet', 'i_progress.txt', "Inference", 0, 100)
    progressIndicator('results/GPU/mobilenet', 'post_progress.txt', "Rendering", 0, 100)

#### - Inferencing using model: ssd300

In [ ]:
#Submit job to the queue
job_id_gpu = !qsub object_detection.sh -l nodes=1:idc001skl:intel-hd-530 -F "-r results/GPU/ssd300 -d GPU -i $VIDEO -m models/SSD300/FP32/VGG_VOC0712Plus_SSD_300x300_ft_iter_160000.xml -n 4" -N obj_det_gpu 
print(job_id_gpu[0]) 
#Progress indicators
if job_id_gpu:
    progressIndicator('results/GPU/ssd300', 'pre_progress.txt', "Preprocessing", 0, 100)
    progressIndicator('results/GPU/ssd300', 'i_progress.txt', "Inference", 0, 100)
    progressIndicator('results/GPU/ssd300', 'post_progress.txt', "Rendering", 0, 100)

#### - Inferencing using model: ssd512

In [ ]:
#Submit job to the queue
job_id_gpu = !qsub object_detection.sh -l nodes=1:idc001skl:intel-hd-530 -F "-r results/GPU/ssd512 -d GPU -i $VIDEO -m models/SSD512/FP32/VGG_VOC0712Plus_SSD_512x512_iter_240000.xml -n 4" -N obj_det_gpu 
print(job_id_gpu[0]) 
#Progress indicators
if job_id_gpu:
    progressIndicator('results/GPU/ssd512', 'pre_progress.txt', "Preprocessing", 0, 100)
    progressIndicator('results/GPU/ssd512', 'i_progress.txt', "Inference", 0, 100)
    progressIndicator('results/GPU/ssd512', 'post_progress.txt', "Rendering", 0, 100)


    
### c) Intel® Movidius™ Neural Compute Stick

#### - Inferencing using **mobilenet-ssd** model

In [ ]:
#Submit job to the queue
job_id_ncs2 = !qsub object_detection.sh -l nodes=1:idc004nc2:intel-ncs2 -F "-r results/ncs2/mobilenet -d MYRIAD -i $VIDEO -m models/mobilenet-ssd/FP16/mobilenet-ssd.xml -n 2" -N obj_det_ncs2
print(job_id_ncs2[0]) 
#Progress indicators
if job_id_ncs2:
    progressIndicator('results/ncs2/mobilenet', 'pre_progress.txt', "Preprocessing", 0, 100)
    progressIndicator('results/ncs2/mobilenet', 'i_progress.txt', "Inference", 0, 100)
    progressIndicator('results/ncs2/mobilenet', 'post_progress.txt', "Rendering", 0, 100)

#### - Inferencing using model: ssd300

In [ ]:
#Submit job to the queue
job_id_ncs2 = !qsub object_detection.sh -l nodes=1:idc004nc2:intel-ncs2 -F "-r results/ncs2/ssd300 -d MYRIAD -i $VIDEO -m models/SSD300/FP16/VGG_VOC0712Plus_SSD_300x300_ft_iter_160000.xml -n 8" -N obj_det_ncs2
print(job_id_ncs2[0]) 
#Progress indicators
if job_id_ncs2:
    progressIndicator('results/ncs2/ssd300', 'pre_progress.txt', "Preprocessing", 0, 100)
    progressIndicator('results/ncs2/ssd300', 'i_progress.txt', "Inference", 0, 100)
    progressIndicator('results/ncs2/ssd300', 'post_progress.txt', "Rendering", 0, 100)

#### - Inferencing using model: ssd512

In [ ]:
#Submit job to the queue
job_id_ncs2 = !qsub object_detection.sh -l nodes=1:idc004nc2:intel-ncs2 -F "-r results/ncs2/ssd512 -d MYRIAD -i $VIDEO -m models/SSD512/FP16/VGG_VOC0712Plus_SSD_512x512_iter_240000.xml -n 8" -N obj_det_ncs2
print(job_id_ncs2[0]) 
#Progress indicators
if job_id_ncs2:
    progressIndicator('results/ncs2/ssd512', 'pre_progress.txt', "Preprocessing", 0, 100)
    progressIndicator('results/ncs2/ssd512', 'i_progress.txt', "Inference", 0, 100)
    progressIndicator('results/ncs2/ssd512', 'post_progress.txt', "Rendering", 0, 100)

### 4. Use the right data type for your target hardware and accuracy needs

In this section, we will consider an example running on a GPU. FP16 operations are better optimized than FP32 on GPUs. We will run the object detection example with SSD models with data types FP16 and FP32 and observe the performance difference.

In [ ]:
#Submit job to the queue
job_id_gpu = !qsub object_detection.sh -l nodes=1:idc001skl:intel-hd-530 -F "-r results/GPU/3/FP32 -d GPU -i $VIDEO -m models/mobilenet-ssd/FP32/mobilenet-ssd.xml -n 4" -N obj_det_gpu 
print(job_id_gpu[0]) 
#Progress indicators
if job_id_gpu:
    progressIndicator('results/GPU/3/FP32', 'pre_progress.txt', "Preprocessing", 0, 100)
    progressIndicator('results/GPU/3/FP32', 'i_progress.txt', "Inference", 0, 100)
    progressIndicator('results/GPU/3/FP32', 'post_progress.txt', "Rendering", 0, 100)

In [ ]:
#Submit job to the queue
job_id_gpu = !qsub object_detection.sh -l nodes=1:idc001skl:intel-hd-530 -F "-r results/GPU/3/FP16 -d GPU -i $VIDEO -m models/mobilenet-ssd/FP16/mobilenet-ssd.xml -n 4" -N obj_det_gpu 
print(job_id_gpu[0]) 
#Progress indicators
if job_id_gpu:
    progressIndicator('results/GPU/3/FP16', 'pre_progress.txt', "Preprocessing", 0, 100)
    progressIndicator('results/GPU/3/FP16', 'i_progress.txt', "Inference", 0, 100)
    progressIndicator('results/GPU/3/FP16', 'post_progress.txt', "Rendering", 0, 100)


It is clear that we got better performance with FP16 models.

### 5. Use async

The async API can improve the overall frame rate of the application. While the accelerator is busy with running inference operations, the application can continue encoding, decoding or post inference data processing on the host. For this section, we will use the object_detection_demo_ssd_async sample. This sample makes asynchronous requests to the inference engine. This reduces the inference request latency, so that the overall framerate is determined by the MAXIMUM(detection time, input capturing time) and not the SUM(detection time, input capturing time).

#### a) Run the async example

In [ ]:
! ./object_detection_demo_ssd_async -i car-detection.mp4 -m models/mobilenet-ssd/FP32/mobilenet-ssd.xml

In [ ]:
videoHTML('IEI Tank (Intel  CPU)', 
         ['obj_async_output.mp4'])

#### b) Run the sync example

In [ ]:
!./object_detection_demo_ssd_sync -i car-detection.mp4 -m models/mobilenet-ssd/FP32/mobilenet-ssd.xml

In [ ]:
videoHTML('IEI Tank (Intel  CPU)', 
          ['obj_sync_output.mp4'])